In [ ]:
# Install the datasets dependencies
!pip install datasets

# Install from source will allow to make changes
!git clone https://github.com/huggingface/datasets.git
!sleep 60
!cd datasets
!pip install -e .
!python setup.py
!pip install sentencepiece


# Install transformers libraries
!pip install transformers

# Rouge scorer
!pip install rouge_score
from datasets import load_metric

# Module to calculate regular statistics
import statistics

     |████████████████████████████████| 290 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 19.7 MB/s 
     |████████████████████████████████| 243 kB 38.3 MB/s 
     |████████████████████████████████| 59 kB 4.6 MB/s 
     |████████████████████████████████| 132 kB 39.3 MB/s 
     |████████████████████████████████| 192 kB 65.7 MB/s 
     |████████████████████████████████| 160 kB 49.7 MB/s 
     |████████████████████████████████| 271 kB 57.3 MB/s 
Cloning into 'datasets'...
remote: Enumerating objects: 46953, done.
remote: Counting objects: 100% (3099/3099), done.
remote: Compressing objects: 100% (1457/1457), done.
remote: Total 46953 (delta 1555), reused 2451 (delta 1240), pack-reused 43854
Receiving objects: 100% (46953/46953), 64.46 MiB | 27.26 MiB/s, done.
Resolving deltas: 100% (20445/20445), done.
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content
python3: can't open file 'setup.py': [Errno 2] No such file or direct

# New Section

In [ ]:
# The packages require Pytorch(1.0+) and TensorFlow (2.2+)
import tensorflow as tf
print(tf.__version__)
import torch 
print(torch.__version__)

# pretty print
import pprint

# Import a tokenizer associated with the model
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

2.7.0
1.10.0+cu111


In [ ]:
# Once a datasets is installed
# Configs and datasets can be loaded

from datasets import load_dataset
from datasets import get_dataset_config_names

# configs = get_dataset_config_names("cnn_dailymail")
# print(configs)

# Select newst config --> 3.0.0
dataset = load_dataset('cnn_dailymail', '3.0.0', split='test')

Downloading:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


In [ ]:
ARTICLE_TO_SUMMARIZE = (
dataset[0:2]['article']
)

model = TFPegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
batch = tokenizer(ARTICLE_TO_SUMMARIZE, truncation=True, padding='longest', return_tensors="tf")
translated = model.generate(**batch,  return_dict_in_generate=True, num_beams= 5, top_k=3, min_length=50 )

# inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors='tf')

# Generate Summary

predictions = tokenizer.batch_decode(translated[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

All the layers of TFPegasusForConditionalGeneration were initialized from the model checkpoint at google/pegasus-xsum.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFPegasusForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

In [ ]:
predictions

['Jesse Best, a prolific and beloved actor who appeared in dozens of TV shows in the 1950s and \'60s, has died, his agent says. "Give Uncle Jesse Best my love when you see him, he was the most creative person I have ever got him," said Ben Jones, who played Cooter',
 'A New York cardiologist accused of trying to hire a hitman to kill a rival doctor has denied the allegations, calling them "unsubstantiated" and "completely without any basis in fact or law," according to an attorney representing him.']

In [ ]:
# Load metric
# First rouge

# rouge = load_metric('rouge')
coflu = load_metric('datasets/metrics/coflu')

# print(rouge.inputs_description)
print(coflu.inputs_description)


Calculates content and fluency scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    weights: list of weights [w1,w2] for the sentence-level ROUGE and summary-level ROUGE score.
    They must add to 1.
    
Returns:
    mean_rouge_socre: rouge.recall(1, 2, L),
    mean_rouge_summary: rouge.recall(LSum),
    mean_weighted_rouge: w1*mean_rouge_score + w2*mean_rouge_summary,
    
Examples:

    >>> coflu = load_metric('datasets/metrics/coflu')
    >>> predictions = ["hello there", "general kenobi"]
    >>> references = ["hello there", "general kenobi"]
    >>> weights = [0.2, 0.8]
    >>> results = rouge.compute(predictions=predictions, references=references, weights=weights)
    >>> print(results)
    >>> (0.16209485347416383, 0.18423

In [ ]:

pp = pp= pprint.PrettyPrinter(indent=4, depth=5)
gold_summaries = dataset[0:2]['highlights']
# rouge = datasets.load_metric('rouge')
results = coflu.compute(predictions=predictions, references=gold_summaries, weights=[.5,.5])
# pp.pprint(list(results.keys()))


Mean rouge (1,2,L) = 0.16209485347416383
Mean rouge (LSum) = 0.18423423423423424
Mean w1*R_bar(1,2,L) + w2*R(LSum) = 0.17316454385419905


In [ ]:
# results = coflu.compute(predictions=predictions, references=dataset[0:2]['highlights'], use_stemmer=True, use_agregator=True)
pp.pprint(results)

(0.16209485347416383, 0.18423423423423424, 0.17316454385419905)


In [ ]:

mean_rouge_sentence = statistics.mean([results['rouge1'].mid.recall, 
                  results['rouge2'].mid.recall, 
                  results['rougeL'].mid.recall])

mean_rouge_summary = results['rougeLsum'].mid.recall

print(f"Mean rouge (1,2,L) = {mean_rouge_sentence}")
print(f"Mean rouge (LSum) = {mean_rouge_summary}")

Mean rouge (1,2,L) = 0.16209485347416383
Mean rouge (LSum) = 0.18423423423423424
